In [28]:
from unfolding_helper import *
from shapely.geometry import *

def get_unfolding(mesh, cutted_faces):
    tree = node(cutted_faces[0])
    tree.add_child(cutted_faces[1])
    return tree.unfold(mesh)

In [29]:
mesh = Mesh('cube.obj')
    
#list of faces on the shortest path
cutted_faces = [5, 2]
    
end_point_id = 2
starting_point_3d = (.5, .5, .1)
    
#unfold the faces from cutted_faces as shown in the previous lectures
unfold = get_unfolding(mesh, cutted_faces)
    
#list of edge_ids that possibly are being cut by the shortest path
cutted_edges = get_cutted_edges(mesh, cutted_faces)
    
#create a Hashmap that maps vertex_id to 2d coordinates
points_2d = map_id_2d_points(mesh, unfold, cutted_faces)
    
#now we can get the 2d projection of the end point
end_point_2d = points_2d[end_point_id]
    
#project starting point from 3d to 2d space using the first Matrix Multiplication in the unfold function
starting_point_2d = starting_point_3d_2_2d(mesh, cutted_faces[0], starting_point_3d)

#List of edges represented as vertex_ids
#Example: [(0, 1), (1, 2), (2, 3), (3, 0), (0, 4), (4, 5), (5, 1), (5, 6), (6, 2), (6, 7), (7, 3), (7, 4)]
#now we can get the vertecies of an edge by edges_as_vertex_ids[edge_id]
edges_as_vertex_ids = [tuple(e) for e in mesh.edge_vertex_indices()]


#list to store generated lines
lines = []

#for each edge_id we get the right vertex_ids 
for e in cutted_edges:
    vertex_ids = edges_as_vertex_ids[e]
    
    #Create a LineString out of 2d coordinates of these verticies
    nl = LineString((points_2d[vertex_ids[0]], points_2d[vertex_ids[1]]))
    
    #Append this line to lines
    lines.append(nl)

# Create a LineString out of the 2d starting and the 2d end point
main_line = LineString((starting_point_2d, end_point_2d))

In [30]:
# Simple Vector Class
class Vec:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def len(self):
        return sqrt(self.x**2 + self.y**2 + self.z**2)

    def normalize(self):
        l = self.len()
        return Vec(self.x/l, self.y/l, self.z/l)

    def __add__(self, v):
        return Vec(self.x + v.x, self.y + v.y, self.z + v.z)

    def __mul__(self, s):
        return Vec(self.x * s, self.y * s, self.z * s)

    def __sub__(self, v):
        return Vec(self.x - v.x, self.y - v.y, self.z - v.z)

In [31]:
class Intersection_Vertex:
    def __init__(self, edge_id, distance):
        self.edge_id = edge_id
        self.distance = distance
    
    def calc_3d_coordinates(self, edge):
        #expecting the edge to already be in Vector form
        return (edge[1] - edge[0]).normalize() * (self.distance) + edge[0]

In [32]:
new_vertices = []
for edge_id, l in enumerate(lines):   
    #intersect mainline with current line l
    p = main_line.intersection(l)  
    
    #if there is an intersection get the coordinates of p and store them
    if p: 
        #turn p into a vector
        p = tuple(p.coords)[0]
        p = Vec(p[0], p[1], 0)
        
        #turn the first point in the line into a vector
        A = tuple(l.coords)[0]
        A = Vec(A[0], A[1], 0)
        
        #calculate the distance betwenn p and A
        dist = (p - A).len()
        
        #turn p into an Intersection_Vertex
        p = Intersection_Vertex(edge_id, dist)
        new_vertices.append(p)

In [33]:
#List of 3d Coordinates for every vertex in the mesh
# use "points_3d[vertex_id]" to get the coordinates of a certain point

points_3d = [tuple(p) for p in mesh.points()]
# TODO: Calculate the 3d position for every Intersection_Vertex in new_vertices
#     -> use "edge_as_vertex_ids" to get the corresponing vertex_ids to any given edge_id
#     -> use "Intersection_Vertex.calc_3d_coordinates(edge)" 
